## Proofs for equisat/non-equisat for variable substitution technique used in magicXform

In the file represented original transition system (ts0) and transformed transition system (ts1)

In [2]:
import sys
sys.path.insert(1, '/Users/ekvashyn/Code/spacer-on-jupyter/src/')
from spacer_tutorial import *
import z3
z3.set_param(proof=True)
z3.set_param(model=True)
z3.set_html_mode(True)


In [3]:
def mk_ts0():
    T = Ts('Ts0')
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    T.Init = z3.And(x == 0, y == 5000)
    T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x >= 5000, y+1, y))
    T.Bad = z3.And(x == 10000, x != y)
    return T

ts0 = mk_ts0()
HtmlStr(ts0)

"Transition System: Ts0<br/>\tInit: x = 0 &and; y = 5000<br/>\tBad: x = 10000 &and; x &ne; y<br/>\tTr: x' = x + 1 &and; y' = If(x &ge; 5000, y + 1, y)"

In [4]:
def mk_ts1():
    T = Ts('Ts1')
    x, x_out = T.add_var(z3.IntSort(), name='x')
    y, y_out = T.add_var(z3.IntSort(), name='y')
    a, a_out = T.add_var(z3.IntSort(), name='a')
    b, b_out = T.add_var(z3.IntSort(), name='b')
    T.Init = z3.And(a == 5000, b == 10000, x == 0, y == a)
    T.Tr = z3.And(x_out == x + 1, y_out == z3.If(x >= a, y+1, y), a_out == a, b_out == b)
    T.Bad = z3.And(x == b, x != y)
    return T

ts1 = mk_ts1()
HtmlStr(ts1)

"Transition System: Ts1<br/>\tInit: a = 5000 &and; b = 10000 &and; x = 0 &and; y = a<br/>\tBad: x = b &and; x &ne; y<br/>\tTr: x' = x + 1 &and; y' = If(x &ge; a, y + 1, y) &and; a' = a &and; b' = b"

In [5]:
def vc_gen(T):
    '''Verification Condition (VC) for an Inductive Invariant'''
    Inv = z3.Function('Inv', *(T.sig() + [z3.BoolSort()]))

    InvPre = Inv(*T.pre_vars())
    InvPost = Inv(*T.post_vars())

    all_vars = T.all()
    vc_init = z3.ForAll(all_vars, z3.Implies(T.Init, InvPre))
    vc_ind = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Tr), InvPost))
    vc_bad = z3.ForAll(all_vars, z3.Implies(z3.And(InvPre, T.Bad), z3.BoolVal(False)))
    return [vc_init, vc_ind, vc_bad], InvPre

In [6]:
vc0, inv0 = vc_gen(ts0)
vc1, inv1 = vc_gen(ts1)

In [7]:
chc_to_str(vc0)

"&forall;x, y, x', y' : x = 0 &and; y = 5000 &rArr; Inv(x, y) <br/><br/>&forall;x, y, x', y' :<br/> Inv(x, y) &and; x' = x + 1 &and; y' = If(x &ge; 5000, y + 1, y) &rArr;<br/> Inv(x', y') <br/><br/>&forall;x, y, x', y' : Inv(x, y) &and; x = 10000 &and; x &ne; y &rArr; False <br/><br/>"

In [8]:
chc_to_str(vc1)

"&forall;x, y, a, b, x', y', a', b' :<br/> a = 5000 &and; b = 10000 &and; x = 0 &and; y = a &rArr; Inv(x, y, a, b) <br/><br/>&forall;x, y, a, b, x', y', a', b' :<br/> Inv(x, y, a, b) &and;<br/> x' = x + 1 &and;<br/> y' = If(x &ge; a, y + 1, y) &and;<br/> a' = a &and;<br/> b' = b &rArr;<br/> Inv(x', y', a', b') <br/><br/>&forall;x, y, a, b, x', y', a', b' :<br/> Inv(x, y, a, b) &and; x = b &and; x &ne; y &rArr; False <br/><br/>"

#### Invariants for those 2 systems locates below

In [9]:
HtmlStr(inv0)

'Inv(x, y)'

In [10]:
HtmlStr(inv1)

'Inv(x, y, a, b)'

In [37]:
res0, answer0 = solve_horn(vc0, max_unfold=100)

In [38]:
res1, answer1 = solve_horn(vc1, max_unfold=100)

In [39]:
res0

<b>sat</b>

In [40]:
res1

<b>sat</b>

In [50]:
answer0

[Inv = [else &rarr;
        (&not;(&#957;<sub>0</sub> &le; 5000) &or; &not;(&#957;<sub>1</sub> &ge; 5001)) &and;
        &not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>1</sub> &ge; 1) &and;
        (&not;(&#957;<sub>0</sub> &ge; 5000) &or; &not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>1</sub> &le; -1)) &and;
        &not;(&#957;<sub>1</sub> &le; 4999)]]

In [51]:
answer1

[Inv = [else &rarr;
        (&not;(&#957;<sub>1</sub> + -1&middot;&#957;<sub>0</sub> &le; -1) &or; &not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>3</sub> &ge; -3)) &and;
        &not;(&#957;<sub>2</sub> + -1&middot;&#957;<sub>3</sub> &ge; -4999) &and;
        (&not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>2</sub> &le; -1) &or; &not;(&#957;<sub>1</sub> + -1&middot;&#957;<sub>2</sub> &ge; 1)) &and;
        (&not;(&#957;<sub>1</sub> + -1&middot;&#957;<sub>0</sub> &le; -1) &or;
         &not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>3</sub> &le; -2) &or;
         &not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>2</sub> &ge; 0)) &and;
        &not;(&#957;<sub>1</sub> + -1&middot;&#957;<sub>2</sub> &le; -1) &and;
        (&not;(&#957;<sub>0</sub> + -1&middot;&#957;<sub>2</sub> &ge; 0) &or; &not;(&#957;<sub>1</sub> + -1&middot;&#957;<sub>0</sub> &ge; 1))]]

In [52]:
answer0.eval(inv0)

(&not;(x &le; 5000) &or; &not;(y &ge; 5001)) &and;
&not;(x + -1&middot;y &ge; 1) &and;
(&not;(x &ge; 5000) &or; &not;(x + -1&middot;y &le; -1)) &and;
&not;(y &le; 4999)

In [49]:
answer1.eval(inv1)

(&not;(y + -1&middot;x &le; -1) &or; &not;(x + -1&middot;b &ge; -3)) &and;
&not;(a + -1&middot;b &ge; -4999) &and;
(&not;(x + -1&middot;a &le; -1) &or; &not;(y + -1&middot;a &ge; 1)) &and;
(&not;(y + -1&middot;x &le; -1) &or; &not;(x + -1&middot;a &ge; 0) &or; &not;(x + -1&middot;b &le; -2)) &and;
&not;(y + -1&middot;a &le; -1) &and;
(&not;(x + -1&middot;a &ge; 0) &or; &not;(y + -1&middot;x &ge; 1))

### 1. Provide cx for the statement: inv2(x,y,a,b) = inv(x,y)[5000->a, 10000->b]

* Invariant for the original benchmark is: <br>
Inv1(x,y) = 

&emsp;(¬(y ≥ 5001) ∨ ¬(y + -1·x ≥ 1)) ∧ ¬(y + -1·x ≤ -1) ∧ ¬(y ≤ 4999) =

&emsp;(y ≥ 5001) => (y ≥ x + 1) ∧ y ≥ x ∧ y > 4999

* Invariant for the transformed benchmark is: <br>
Inv2(x,y,a,b) = 

&emsp;(¬(y ≥ 5001) ∨ x ≥ y) ∧ y ≥ a ∧ x ≤ y =

&emsp; (y > a => x ≥ y) ∧ y ≥ a ∧ x ≤ y


#### We need to prove that Inv1(x,y)[5000->a, 10000->b] != Inv2(x,y,a,b) and provide a cx


Let's rewrite Inv1(x,y) in such way: <br>
Inv1(x,y) = ((y ≥ 5001 => x>=y) ∧ y ≥ 5000 ∧ x ≤ y)
<br>
Inv1(x,y)[5000->a, 10000->b] = ((y ≥ 5001 => x≥y) ∧ y ≥ a ∧ x ≤ y)

#### Let's to find a cx using z3:

In [56]:
from z3 import *

# Create Z3 variables
a, b, x, y, x_prime, y_prime = Ints('a b x y x_prime y_prime')

# Create a Z3 solver
solver = Solver()

# Initial conditions
solver.add(x == 0)
solver.add(y == a)

# Transition relation constraints
transition_constraints = And(
    x_prime == x + 1,
    y_prime == If(x >= a, y + 1, y)
)

# Invariant constraints
invariant_constraints = And(
    Implies(y > 5001, x >= y),
    y >= a,
    x <= y
)

# Add transition relation and invariant constraints to the solver
solver.add(transition_constraints)
solver.add(Not(invariant_constraints))  # Adding negation of the invariant for checking

# Check for satisfiability
if solver.check() == unsat:
    print("Invariant holds for the transition system.")
else:
    print("Invariant does not hold for the transition system.")
    counterexample = solver.model()
    print(f"Counterexample found:")
    print("x =", counterexample[x])
    print("y =", counterexample[y])
    print("a =", counterexample[a])


Invariant does not hold for the transition system.
Counterexample found:
x = 0
y = 5002
a = 5002


### 2. Prove the statement:
inv2(x,y,a,b) = inv(x,y) ∧ a = 5000 ∧ b = 10000

In [57]:
# vars for Ts0
x0, y0 = Ints('x0 y0')

# vars for Ts1
x1, y1, a1, b1 = Ints('x1 y1 a1 b1')

solver = Solver()

# Define the invariants for Ts0 and Ts1
I0 = And(
    Implies(y > 5000, x >= y),
    y >= 5000,
    x <= y
)
I1 = And( a1 == 5000, b1 == 10000, Implies(y1 >= a, x1 >= y1), x1 <= y1, y1 >= a1)

# Check if (I0 and (a = 5000) and (b = 10000)) is equivalent to I1
solver.add(Not(Implies(And(I0, a1 == 5000, b1 == 10000), I1)))
solver.add(Not(Implies(I1, And(I0, a1 == 5000, b1 == 10000))))

# Check for satisfiability (unsat implies equivalence)
if solver.check() == unsat:
    print("Invariant equivalency holds.")
else:
    print("Invariant equivalency does not hold.")


Invariant equivalency holds.
